In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
# #Uzupełnienie kolumn zawierających puste pola wartością ?
# loan_data[["emp_length", 
#            "mths_since_last_delinq", 
#            "pub_rec_bankruptcies",
#           "revol_util"]] = loan_data[["emp_length", 
#                                                  "mths_since_last_delinq", 
#                                                  "pub_rec_bankruptcies",
#                                      "revol_util"]].fillna(0)

In [ ]:
# loan_data['verification_status'].replace({'Verified':1, 
#                                           'Source Verified':1, 
#                                           'Not Verified':0}, inplace = True)
# loan_data['loan_status'].replace({'Fully Paid':1, 
#                                   'Charged Off':0}, inplace = True)

In [ ]:
# #Przekształcamy ocenę z kategorycznej do numerycznej kolumny (A1 -> 1.1)
# loan_data['sub_grade'] = loan_data['sub_grade'].str[1:]
# loan_data['sub_grade'] = pd.to_numeric(loan_data['sub_grade'])
# #Ocena numeryczna
# loan_data['numeric_grade'] = loan_data['grade']+ loan_data['sub_grade']/10
# loan_data.drop(['sub_grade', 
#                 'grade'], axis=1, inplace = True)

In [ ]:
#print('Number of outliers in df: ',concrete_df1[((concrete_df1.cement - concrete_df1.cement.mean()) / concrete_df1.cement.std()).abs() >3]['cement'].count())


In [ ]:
#Zmieniamy kolumnę 'home_ownership' na kategoryczną za pomocą funkcji get_dummies
#loan_data = pd.get_dummies(loan_data, columns=['home_ownership'])

In [ ]:
#loan_data['years_of_credit_history'] = round(((loan_data['issue_d'] - loan_data['earliest_cr_line']).dt.days/365), 1)
#loan_data.drop(['earliest_cr_line'], axis=1, inplace = True)